In [1]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from TransformerGlow import AdamWeightDecayOptimizer, FactorTrainer
import numpy as np
import torch
import random
import pandas as pd
import pickle
# import os, sys
# import time
# import datetime
# import torch.nn.functional as F
# from torch import nn



# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A40


In [2]:
def get_examples(embeddings, gen_labels, attribute_ids, logits, rand_seed=42 , train=True):
    random.seed(rand_seed)
    female_embeddings = []
    male_embeddings = []
    female_gen_labels = []
    male_gen_labels = []
    female_attribute_ids = []
    male_attribute_ids = []
    
    for key,i in enumerate(gen_labels):
        if i==0:
            # print(logits[key])
            if logits[key]<-0:
                female_embeddings.append(embeddings[key])
                female_gen_labels.append(i)
                female_attribute_ids.append(attribute_ids[key])
                # print(key)
        else:
            if logits[key]>0:
                male_embeddings.append(embeddings[key])
                male_gen_labels.append(i)
                male_attribute_ids.append(attribute_ids[key])

    random.shuffle(female_embeddings)
    random.shuffle(male_embeddings)
    female_embeddings_1 = female_embeddings[:int(len(female_embeddings)*0.5)].copy()
    male_embeddings_1 = male_embeddings[:int(len(male_embeddings)*0.5)].copy()
    female_embeddings_2 = female_embeddings[int(len(female_embeddings)*0.5):].copy()
    male_embeddings_2 = male_embeddings[int(len(male_embeddings)*0.5):].copy()
    # random.Random(rand_seed).shuffle(female_embeddings_2)
    # random.Random(rand_seed).shuffle(male_embeddings_2)

    female_data_pairs = [[x, y] for x, y in zip(female_embeddings_1, female_embeddings_2)]
    male_data_pairs = [[x, y] for x, y in zip(male_embeddings_1, male_embeddings_2)]

    gender_data_pairs = female_data_pairs + male_data_pairs
    random.shuffle(gender_data_pairs)

    
    

    if train==True:
        gender_data_pairs = torch.tensor(np.array(gender_data_pairs))
        return gender_data_pairs

    else:
        female_data_pairs_test = torch.tensor(np.array(female_data_pairs))
        male_data_pairs_test = torch.tensor(np.array(male_data_pairs))
        return female_data_pairs_test, male_data_pairs_test

In [3]:
# import random
# train_limit = int(0.99*len(embeddings))
# random.Random(4).shuffle(embeddings)
# random.Random(4).shuffle(gen_labels)
# random.Random(4).shuffle(attribute_ids)
# train_embeddings = embeddings[:train_limit]
# train_gen_labels = gen_labels[:train_limit]
# train_attribute_ids = attribute_ids[:train_limit]

# test_embeddings = embeddings[train_limit:]
# test_gen_labels = gen_labels[train_limit:]
# test_attribute_ids = attribute_ids[train_limit:]



In [4]:
# import importlib
# importlib.reload(FactorTrainer)

# TransformerGlow = importlib.reload(FactorTrainer)
# FactorTrainer = reload(FactorTrainer)

FactorTrainer_config = {
  "n_factors": 10,
  # "factor_dim":211,
  "in_channel": 768,
  "n_flow": 6,
  "hidden_depth": 2,
  "hidden_dim": 100,
# "rho": 0.999
  "rho": 0.98
}

bertflow = FactorTrainer(FactorTrainer_config).cuda()

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters= [
        {
            "params": [p for n, p in bertflow.glow.named_parameters()  \
                            if not any(nd in n for nd in no_decay)],  # Note only the parameters within bertflow.glow will be updated and the Transformer will be freezed during training.
            "weight_decay": 0.01,
        },
        {
            "params": [p for n, p in bertflow.glow.named_parameters()  \
                            if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]



optimizer = AdamWeightDecayOptimizer(
        params=optimizer_grouped_parameters, 
        lr=1e-3, 
        eps=1e-6,
    )

In [5]:
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/embeddings.pkl', 'rb') as f:
#         embeddings = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/gender.pkl', 'rb') as f:
#         gen_labels = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/token_ids.pkl', 'rb') as f:
#         attribute_ids = pickle.load(f)

######### TRAIN MODEL ################

In [6]:
import os
attribute_pair = "african_european"
model_path = "bertflow_model_"+attribute_pair
if not os.path.exists(model_path):
    os.makedirs(model_path)


bertflow.train()

# for jj in range(1,2):
for iteration in range(2000,13000, 2000):
    print(iteration)

    with open('../1_viz/attribute_embeddings_'+attribute_pair+'/embeddings_'+str(iteration)+'.pkl', 'rb') as f:
        embeddings = pickle.load(f)
    with open('../1_viz/attribute_embeddings_'+attribute_pair+'/gender_'+str(iteration)+'.pkl', 'rb') as f:
            gen_labels = pickle.load(f)
    with open('../1_viz/attribute_embeddings_'+attribute_pair+'/token_ids_'+str(iteration)+'.pkl', 'rb') as f:
            attribute_ids = pickle.load(f)
    with open('../1_viz/attribute_embeddings_'+attribute_pair+'/logits_'+str(iteration)+'.pkl', 'rb') as f:
            logits = pickle.load(f)
    # with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/embeddings_'+str(iteration)+'.pkl', 'rb') as f:
    #     embeddings = pickle.load(f)
    # with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/gender_'+str(iteration)+'.pkl', 'rb') as f:
    #         gen_labels = pickle.load(f)
    # with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/token_ids_'+str(iteration)+'.pkl', 'rb') as f:
    #         attribute_ids = pickle.load(f)
    # # with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/logits_'+str(iteration)+'.pkl', 'rb') as f:
    # #         logits = pickle.load(f)

    for epoch in range(4):
        # reload data shuffled
        train_gender_data_pairs = get_examples(embeddings, gen_labels, attribute_ids, logits, rand_seed=epoch)
        
        
        batch_size = 32
        
        
        
        # Create the DataLoaders for our training and validation sets.
        # We'll take training samples in random order. 
        train_dataloader = DataLoader(
                    train_gender_data_pairs,  # The training samples.
                    sampler = RandomSampler(train_gender_data_pairs), # Select batches randomly
                    batch_size = batch_size # Trains with this batch size.
                )
    
       
        print("epoch:", epoch)
        for step, batch in enumerate(train_dataloader):
            # print(step)
            # print(batch.shape)
            # bertflow.train()
            z, loss = bertflow(batch.to(device))  # Here z is the sentence embedding
            if step%100==0:
                print(loss)
                print(loss, file=open('output.txt', 'a'))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
        # if (iteration)%2000==0:
        # # if (epoch+1)%4==0:
        #     torch.save(bertflow, model_path+'/bertflow_rho_0999_factors_6_iterations_'+str(iteration)+'.pth')
    torch.save(evalbertflow, model_path+'/bertflow_rho_0999_factors_'+n_factors+'.pth')

36997
epoch: 0
tensor(1871.3604, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-424.0946, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-540.9125, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-657.1843, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-608.1889, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-667.6589, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-719.4258, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-667.8638, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-749.1887, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-803.5557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-799.0875, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-797.7902, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-761.2063, device='cuda:0', grad_fn=<AddBackward0>)
epoch: 1
tensor(-768.5020, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-854.0918, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-801.3319, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-839.3717, device='cuda:0

In [7]:
# import os
# attribute_pair = "asian_european"
# model_path = "bertflow_model_"+attribute_pair
# if not os.path.exists(model_path):
#     os.makedirs(model_path)


# bertflow.train()

# for jj in range(1,2):
#     for iteration in range(1500,31500,1500):
#         print(iteration)
    
#         # with open('../1_viz/attribute_embeddings/embeddings_1000.pkl', 'rb') as f:
#         #     embeddings = pickle.load(f)
#         # with open('../1_viz/attribute_embeddings/gender_1000.pkl', 'rb') as f:
#         #     gen_labels = pickle.load(f)
#         # with open('../1_viz/attribute_embeddings/token_ids_1000.pkl', 'rb') as f:
#         #     attribute_ids = pickle.load(f)
#         with open('../1_viz/attribute_embeddings_'+attribute_pair+'/embeddings_'+str(jj)+'_'+str(iteration)+'.pkl', 'rb') as f:
#             embeddings = pickle.load(f)
#         with open('../1_viz/attribute_embeddings_'+attribute_pair+'/gender_'+str(jj)+'_'+str(iteration)+'.pkl', 'rb') as f:
#                 gen_labels = pickle.load(f)
#         with open('../1_viz/attribute_embeddings_'+attribute_pair+'/token_ids_'+str(jj)+'_'+str(iteration)+'.pkl', 'rb') as f:
#                 attribute_ids = pickle.load(f)
#         with open('../1_viz/attribute_embeddings_'+attribute_pair+'/logits_'+str(jj)+'_'+str(iteration)+'.pkl', 'rb') as f:
#                 logits = pickle.load(f)
    
#         for epoch in range(1):
#             # reload data shuffled
#             train_gender_data_pairs = get_examples(embeddings, gen_labels, attribute_ids, logits, epoch)
            
            
#             batch_size = 32
            
            
            
#             # Create the DataLoaders for our training and validation sets.
#             # We'll take training samples in random order. 
#             train_dataloader = DataLoader(
#                         train_gender_data_pairs,  # The training samples.
#                         sampler = RandomSampler(train_gender_data_pairs), # Select batches randomly
#                         batch_size = batch_size # Trains with this batch size.
#                     )
        
           
#             print("epoch:", epoch)
#             for step, batch in enumerate(train_dataloader):
#                 # print(step)
#                 # print(batch.shape)
#                 # bertflow.train()
#                 z, loss = bertflow(batch.to(device))  # Here z is the sentence embedding
#                 if step%100==0:
#                     print(loss)
#                     print(loss, file=open('output.txt', 'a'))
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()
        
#             # if (iteration)%4500==0:
#             # if (epoch+1)%4==0:
#             torch.save(bertflow, model_path+'/bertflow_rho_0999_factors_6_iterations_'+str(iteration)+'.pth')